In [1]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

DATA_STORE = Path('assets.h5')

import pandas as pd
import gdown
import os

# File path
file_path = 'wiki_prices.csv'

# Download the file from Google Drive only if it doesn't exist
if not os.path.exists(file_path):
    gdown.download('https://drive.google.com/uc?export=download&id=1RTEvR3S_jNZNIn_WldbCKfsDb2pl3KZf', file_path, quiet=False)
else:
    print(f"File '{file_path}' already exists. Skipping download.")

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path,
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True).sort_index()

# Now you can work with the DataFrame 'df'

with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/prices', df)
    
df = pd.read_csv('wiki_stocks.csv')

with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/stocks', df)

df = web.DataReader(name='SP500', data_source='fred', start=2009).squeeze().to_frame('close')
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/fred', df)


# Alternatively, download S&P500 data from stooq.com; at the time of writing the data was available since 1789. 
# You can switch from Polish to English on the lower right-hand side.

# We store the data from 1950-2020:

sp500_stooq = (pd.read_csv('^spx_d.csv', index_col=0,
                     parse_dates=True).loc['1950':'2019'].rename(columns=str.lower))    

with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/stooq', sp500_stooq)

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url, header=0)[0]
df.columns = ['ticker', 'name',  'gics_sector', 'gics_sub_industry',
              'location', 'first_added', 'cik', 'founded']

with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/stocks', df)

df = pd.read_csv('us_equities_meta_data.csv')

with pd.HDFStore(DATA_STORE) as store:
    store.put('us_equities/stocks', df.set_index('ticker'))


File 'wiki_prices.csv' already exists. Skipping download.
